In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import platform
import pathlib  
import os.path
import netCDF4 as nc
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import yaml

from superflexpy.implementation.elements.gr4j import BaseElement, UnitHydrograph1, UnitHydrograph2, FluxAggregator 
from superflexpy.implementation.elements.gr4j import InterceptionFilter, ProductionStore, RoutingStore
from superflexpy.implementation.elements.structure_elements import Transparent, Splitter, Junction
from superflexpy.implementation.root_finders.pegasus import PegasusPython
from superflexpy.implementation.numerical_approximators.implicit_euler import ImplicitEulerPython
from superflexpy.framework.unit import Unit
from superflexpy.framework.node import Node
from superflexpy.framework.network import Network
# from utils.precipitation_evaporation import *

In [23]:
cwd = pathlib.Path().resolve()
src = cwd.parent
data = src.parent.parent.parent
OS_type = platform.system()

if OS_type == 'Darwin':
    username = 'Mats '
    data_path = os.path.join(data, 'data_tana', 'catchments')
    shape_path = os.path.join(data, 'data_tana', 'catchments')
    
else:
    username = 'Mootje'
    data_path = os.path.join(data, 'OneDrive - Delft University of Technology', 'TU Delft', 'Master ENVM', 'MDP', 'Model', 'Data', 'Mini_data')
    connection_path = os.path.join(data, 'Desktop', 'MDP_Github', 'Git_mdp', 'data', 'catchments')
    # shape_path = os.path.join(data, 'OneDrive - Delft University of Technology', 'TU Delft', 'Master ENVM', 'MDP', 'Model', 'Data', 'Shapefiles','Mini_shapes')

print(f"Welcome {username}, have a wondeful day on your {OS_type} machine. Your data should be located in {data_path}")
# print(data)

Welcome Mootje, have a wondeful day on your Windows machine. Your data should be located in C:\Users\Moniek van Zon\OneDrive - Delft University of Technology\TU Delft\Master ENVM\MDP\Model\Data\Mini_data


In [24]:
data_files = glob.glob(os.path.join(data_path, '*.nc'))
print(len(data_files))
datasets = {}
file_identifier_list = []

for file_path in data_files:
    # Extract the file identifier from the file name
    file_name = os.path.basename(file_path)  # Get just the file name
    file_identifier = file_name.split('_')[1]
    file_identifier2 = file_identifier.split('.')[0]
    file_identifier_list.append(file_identifier2)
   
    # Open the dataset
    dataset = xr.open_dataset(file_path)
    # print(dataset)
    # Add the file identifier as a new coordinate
    dataset = dataset.assign_coords(file_identifier=file_identifier)
    
    # Add the dataset to the dictionary with the file identifier as the key
    datasets[file_identifier2] = dataset



263


In [25]:
def GR4J_element(x1,x2,x3,x4, data_set, file_name):
    #x1, x2, x3, x4 = (50.0, 0.1, 20.0, 3.5)

    # name = data_set[file_name]
    name = file_name
    # print(name)
    # name = name.replace('_', '-')  
    area = data_set[file_name]['area_m2'].values[0]
    # print(area)
    precipitation = data_set[file_name]['precipitation'].values
    # print(precipitation)
    evaporation =  data_set[file_name]['average_evap'].values
    # print(evaporation)

    root_finder = PegasusPython()  # Use the default parameters
    numerical_approximation = ImplicitEulerPython(root_finder)

    interception_filter = InterceptionFilter(id='ir')

    production_store = ProductionStore(parameters={'x1': x1, 'alpha': 2.0,
                                                   'beta': 5.0, 'ni': 4/9},
                                       states={'S0': 5.0},
                                       approximation=numerical_approximation,
                                       id='ps')

    splitter = Splitter(weight=[[0.9], [0.1]],
                        direction=[[0], [0]],
                        id='spl')

    unit_hydrograph_1 = UnitHydrograph1(parameters={'lag-time': x4},
                                        states={'lag': None},
                                        id='uh1')

    unit_hydrograph_2 = UnitHydrograph2(parameters={'lag-time': 2*x4},
                                        states={'lag': None},
                                        id='uh2')

    routing_store = RoutingStore(parameters={'x2': x2, 'x3': x3,
                                             'gamma': 5.0, 'omega': 3.5},
                                 states={'S0': 20.0},
                                 approximation=numerical_approximation,
                                 id='rs')

    transparent = Transparent(id='tr')

    junction = Junction(direction=[[0, None],  # First output
                                   [1, None],  # Second output
                                   [None, 0]], # Third output
                        id='jun')

    flux_aggregator = FluxAggregator(id='fa')

    catchment_element = Unit(layers=[[interception_filter],
                     [production_store],
                     [splitter],
                     [unit_hydrograph_1, unit_hydrograph_2],
                     [routing_store, transparent],
                     [junction],
                     [flux_aggregator]],
             id=f'unit-{name}')
    
    
    current_node = Node(
    units=[catchment_element],
    weights = [1.0],
    area=area,
    id=f'node-{name}'
    )
    
    current_node.set_input([evaporation, precipitation])
    
    return current_node

In [26]:
x1, x2, x3, x4 = 50.0, 0.1, 20., 2.4

In [27]:
node_1 = GR4J_element(x1, x2, x3, x4, datasets, '1021')
node_2 = GR4J_element(x1, x2, x3, x4, datasets, '632')
node_3 = GR4J_element(x1, x2, x3, x4, datasets, '144')

In [28]:
# add all the GR4J elements in one dictionary

nodes_dict = {}

for filename in file_identifier_list:
    # Call GR4J_element function for the current filename
    node = GR4J_element(x1, x2, x3, x4, datasets, filename)
    # Add the Node object to the dictionary with filename as key
    nodes_dict[filename] = node


In [29]:
# Open the network file 

connection_file = glob.glob(os.path.join(connection_path, '*.yaml'))

with open(connection_file[0], 'r') as yaml_file:
    full_network = yaml.load(yaml_file, Loader=yaml.FullLoader)
full_network

{'fid_0.gpkg': 'fid_233.gpkg',
 'fid_1.gpkg': 'fid_38.gpkg',
 'fid_10.gpkg': 'fid_229.gpkg',
 'fid_100.gpkg': 'fid_135.gpkg',
 'fid_101.gpkg': 'fid_12.gpkg',
 'fid_102.gpkg': 'fid_225.gpkg',
 'fid_103.gpkg': 'fid_23.gpkg',
 'fid_104.gpkg': 'fid_205.gpkg',
 'fid_105.gpkg': 'fid_17.gpkg',
 'fid_106.gpkg': 'fid_45.gpkg',
 'fid_107.gpkg': 'fid_20.gpkg',
 'fid_109.gpkg': 'fid_12.gpkg',
 'fid_11.gpkg': 'fid_31.gpkg',
 'fid_110.gpkg': 'fid_144.gpkg',
 'fid_111.gpkg': 'fid_101.gpkg',
 'fid_112.gpkg': 'fid_12.gpkg',
 'fid_113.gpkg': 'fid_12.gpkg',
 'fid_114.gpkg': 'fid_205.gpkg',
 'fid_115.gpkg': 'fid_229.gpkg',
 'fid_116.gpkg': 'fid_12.gpkg',
 'fid_117.gpkg': 'fid_113.gpkg',
 'fid_118.gpkg': 'fid_174.gpkg',
 'fid_119.gpkg': 'fid_31.gpkg',
 'fid_12.gpkg': 'fid_144.gpkg',
 'fid_120.gpkg': 'fid_171.gpkg',
 'fid_121.gpkg': 'fid_68.gpkg',
 'fid_122.gpkg': 'fid_253.gpkg',
 'fid_123.gpkg': 'fid_12.gpkg',
 'fid_124.gpkg': 'fid_10.gpkg',
 'fid_125.gpkg': 'fid_1.gpkg',
 'fid_126.gpkg': 'fid_0.gpkg',
 'f

In [14]:
new_full_network = {}
keys_list = []
for key, value in full_network.items():
    new_key = key.split('_')[1].split('.')[0]  # Extract the number from the key
    keys_list.append(new_key)
    new_value = value.split('_')[1].split('.')[0]  # Extract the number from the value
    new_full_network[new_key] = new_value

print(new_full_network)


{'0': '233', '1': '38', '10': '229', '100': '135', '101': '12', '102': '225', '103': '23', '104': '205', '105': '17', '106': '45', '107': '20', '109': '12', '11': '31', '110': '144', '111': '101', '112': '12', '113': '12', '114': '205', '115': '229', '116': '12', '117': '113', '118': '174', '119': '31', '12': '144', '120': '171', '121': '68', '122': '253', '123': '12', '124': '10', '125': '1', '126': '0', '127': '233', '128': '229', '129': '255', '13': '188', '130': '126', '131': '215', '132': '12', '133': '113', '134': '147', '135': '249', '136': '188', '137': '12', '138': '205', '139': '68', '14': '147', '140': '255', '141': '195', '142': '12', '143': '147', '144': '17', '145': '247', '146': '119', '147': '171', '148': '238', '149': '12', '15': '233', '150': '24', '151': '260', '152': '149', '153': '193', '154': '147', '155': '12', '156': '144', '157': '12', '158': '41', '159': '116', '16': '68', '160': '18', '161': '68', '162': '188', '163': '34', '164': '116', '165': '246', '166': 

In [16]:
nodes_gr4j = []

for key in keys_list:
    nodes_gr4j[key] = GR4J_element(x1, x2, x3, x4, dataset, key)

KeyError: "No variable named '0'. Variables on the dataset include ['time', 'precipitation', 'area_m2', 'average_evap', 'file_identifier']"